Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [81]:
NAME = "JEAN GEORGE ALVES EVANGELISTA"
COLLABORATORS = ""

---

# Exercício Prático 5: Decomposição PA=LU

Neste exercício vamos alterar a implementação da decomposição LU vista em sala para incluir a troca da linhas da pivotação parcial.

### Apresentação da Decomposição LU e da Eliminação de Gauss

Na aula do dia 11/04 vimos que a decomposição $A = LU$ consiste em decompor uma matriz $A$ no produto de uma matriz triangular inferior $L$ por uma matriz triangular superior $U$. Em particular, observamos que os multiplicadores da eliminação de Gauss são utilizados para "preencher" a matriz $L$.

**Por simplicidade, vamos assumir que todas as matrizes são quadradas.**

A implementação da decomposição LU é apresentada abaixo.

In [82]:
import numpy as np
def LU(A):
    U = np.copy(A)
    m, n = A.shape
    L = np.zeros((n,n))
    for j in range(n-1):
        for i in range(j+1,n):
            L[i,j] = U[i,j]/U[j,j]
            U[i,j+1:n] = U[i,j+1:n]-L[i,j] * U[j,j+1:n]
    L = L + np.eye(n)
    return L, np.triu(U)

Um dos problemas da decomposição LU sem pivotação é a possibilidade de ocorrerem divisões por zero. Ainda que elas não ocorram, a pivotação parcial traz maior estabilidade numérica (reduz erros de arredondamento intrínsecos à precisão finita) ao dividir sempre por números de grande magnitude.

## Implementação da Decomposição LU com pivotação parcial

Na pivotação parcial, ao invés de escolhermos sempre os elementos da diagonal como pivô, iremos escolher o elemento, da diagonal para baixo, que tiver o maior valor absoluto.

Em sala, vimos como manter um vetor $p$ indicando as permutações de linhas efetuadas durante a pivotação parcial. Desta vez, iremos realmente permutar as linhas da matriz $U$ (cópia de $A$) de lugar.

**1.** Escreva uma função troca(a,b) que troca o conteúdo de dois numpy arrays a e b. Ela não deve retornar nada. (Dica: Lembre-se de que ```a=b``` faz com que as duas variáveis apontem para o mesmo local. Para criar uma cópia de um vetor ```a```, você deve usar ```a.copy()```).

In [83]:
def troca(a,b):
    aux = np.copy(b)
    b[:] = np.copy(a)
    a[:] = np.copy(aux)

In [84]:
# testes
u = np.array([1,2,3])
v = np.array([4,5,6])
troca(u,v)

assert np.all(u == np.array([4,5,6]))
assert np.all(v == np.array([1,2,3]))

In [85]:
# testes ocultos


**2.** Implemente a função encontraIndicePivo(v) que retorna o índice do elemento de maior valor absoluto dado um numpy array $v$. Se quiser, você pode usar as funções np.argmax e np.abs para resolver esta questão.

In [86]:
def encontraIndicePivo(v):
    return np.argmax(np.abs(v), axis = 0)

In [87]:
# testes
a = np.array([1,-2,3])
b = np.array([1,-4,3])

assert encontraIndicePivo(a) == 2
assert encontraIndicePivo(b) == 1

In [88]:
# testes ocultos


**3.** Usando a função LU da Parte 0 como base, mostre como usar as funções que você criou para implementar a decomposição $PA=LU$. A função LUPivot(A) deve retornar L, U e a matriz de permutação P.

*Dica 1:* Note que o índice do pivô na $k$-ésima iteração não é simplesmente encontraIndicePivo(U[k:,k]).

*Dica 2*: Após encontrar o pivô, você deve trocar as linhas correspondentes em L, U e P.

In [89]:
def LUPivot(A):
    U = np.copy(A)
    m,n = A.shape
    L = np.zeros((n,n))
    P = np.eye(n)
    for k in range(n-1):
        pivo = encontraIndicePivo(U[k:,k])+k  
        troca(P[pivo,:], P[k,:])
        troca(U[pivo,:], U[k,:])
        troca(L[pivo,:], L[k,:])
        for i in range(k+1,n):
            L[i,k] = U[i,k]/U[k,k]
            U[i,k+1:n] = U[i,k+1:n]-L[i,k] * U[k,k+1:n]     
    L = L + np.eye(n)
    return L, np.triu(U), P

In [90]:
# testes

A = np.array([[4,-1,0,-1],[1,-2,1,0],[0,4,-4,1],[5,0,5,-1]]).astype(np.float)
L,U,P = LUPivot(A)

### BEGIN TESTS
assert np.allclose(L,np.array([[1,0,0,0],[0,1,0,0],[.8,-.25,1,0],[.2,-.5,.4,1]]))
assert np.allclose(U,np.array([[5,0,5,-1],[0,4,-4,1],[0,0,-5,0.05],[0,0,0,.68]]))
assert np.allclose(P,np.array([[0,0,0,1],[0,0,1,0],[1,0,0,0],[0,1,0,0]]))
### END TESTS

In [91]:
# testes ocultos
